# NOTEBOOK ENTREGABLE 0 SIMPSONS

En este Notebook se realiza la representación de los datos obtenidos en el contenedor de Python sobre los datos obtenidos de la [API de los simpsons][simpsons]


[//]: # (These are reference links used in the body of this note and get stripped out when the markdown processor does its job.)

[simpsons]: <https://thesimpsonsquoteapi.glitch.me/quotes>

## Gráfico palabras más habituales

En el gráfico, se muestra una representación de las palabras más habituales generadas con el programa main.py que se van guardando en el archivo countedWords.csv


In [131]:
import pandas as pd
import time
from matplotlib import pyplot as plt
from IPython import display



# This loops updates the bar graph every second, to run the next cell, this cell should be stopped first
while True:
    data = pd.read_csv('results/countedWords.csv')

    df = pd.DataFrame(data)

    dfSorted = df.sort_values(by='Cuenta', ascending=False)



    X = list(dfSorted.iloc[:20, 0])
    Y = list(dfSorted.iloc[:20, 1])

    # Plot the data using bar() method
    plt.title("20 Palabras más comunes frases Simpsons")
    plt.xlabel("Palabra")
    plt.ylabel("Cuenta")

    # Rotating x-axis words 90º to avoid overlapping
    plt.xticks(rotation=90)
    
    # Defining graph as a bar chart
    plt.bar(X, Y, color='b')
    
    # These two lines update de graph every second for a live uptade of the words
    
    
    display.display(plt.show())
    display.clear_output(wait=True)
    
    time.sleep(2)

KeyboardInterrupt: 

En la siguiente celda se imprime un listado de todos los directorios con nombres de personajes, así como del nombre de la imagen de cada uno de los personajes

In [50]:
import os
import time
import IPython
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from IPython import display



while True:
    for i in os.listdir(r'./results'):
        if '.' not in i:
            print(f'Directorio: {i}')
            for j in os.listdir('./results/'+i):
                if '.png' in j:
                    image = mpimg.imread('./results/'+i+'/'+j)
                    
                    fig = plt.figure(figsize = (2,2))
                    ax3 = fig.add_subplot(111)
                    ax3.imshow(image)
                    plt.axis('off')
                    display.display(plt.show())
                    
    display.clear_output(wait=True)   
    time.sleep(30)            

KeyboardInterrupt: 